<a href="https://colab.research.google.com/github/varun-sirpal10/convolutional-NN/blob/master/Transfer_Learning_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer Learning

In [1]:
!unzip "Images.zip"

Archive:  Images.zip
   creating: Images/cats/
  inflating: Images/cats/cat.1.jpg   
  inflating: Images/cats/cat.10.jpg  
  inflating: Images/cats/cat.100.jpg  
  inflating: Images/cats/cat.101.jpg  
  inflating: Images/cats/cat.102.jpg  
  inflating: Images/cats/cat.103.jpg  
  inflating: Images/cats/cat.104.jpg  
  inflating: Images/cats/cat.105.jpg  
  inflating: Images/cats/cat.106.jpg  
  inflating: Images/cats/cat.107.jpg  
  inflating: Images/cats/cat.108.jpg  
  inflating: Images/cats/cat.109.jpg  
  inflating: Images/cats/cat.11.jpg  
  inflating: Images/cats/cat.110.jpg  
  inflating: Images/cats/cat.111.jpg  
  inflating: Images/cats/cat.112.jpg  
  inflating: Images/cats/cat.113.jpg  
  inflating: Images/cats/cat.114.jpg  
  inflating: Images/cats/cat.115.jpg  
  inflating: Images/cats/cat.116.jpg  
  inflating: Images/cats/cat.117.jpg  
  inflating: Images/cats/cat.118.jpg  
  inflating: Images/cats/cat.119.jpg  
  inflating: Images/cats/cat.12.jpg  
  inflating: Images/c

In [0]:
import os

In [5]:
folders = os.listdir("Images")
print(folders)

['cats', 'horses', 'humans', 'dogs']


In [0]:
image_data = []
labels = []

label_dict = {
    "cats":0,
    "dogs":1,
    "horses":2,
    "humans":3
}

In [0]:
from keras.preprocessing import image

for ix in folders:
  path = os.path.join("Images",ix)
  for im in os.listdir(path):
    img = image.load_img(os.path.join(path,im),target_size=((224,224)))
    img_array = image.img_to_array(img)
    image_data.append(img_array)
    labels.append(label_dict[ix])

In [10]:
print(len(image_data),len(labels))

808 808


In [0]:
import random

combined = list(zip(image_data,labels))
random.shuffle(combined)

image_data[:],labels[:] = zip(*combined)

In [12]:
print(labels[:5])

[2, 1, 0, 1, 2]


In [15]:
import numpy as np

X_train = np.array(image_data)
Y_train = np.array(labels)

print(X_train.shape,Y_train.shape)

(808, 224, 224, 3) (808,)


In [17]:
from keras.utils import np_utils

Y_train = np_utils.to_categorical(Y_train)
print(X_train.shape,Y_train.shape)

(808, 224, 224, 3) (808, 4)


## Create ResNet 50 Model

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model
import matplotlib.pyplot as plt

In [26]:
model = ResNet50(weights="imagenet",include_top=False,input_shape=(224,224,3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [27]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [28]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256,activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(4,activation='softmax')(d1)

model_new = Model(input=model.input,outputs=fc2)
model_new.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  


In [0]:
adam = Adam(lr=0.0003)
model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [30]:
for ix in range(len(model_new.layers)):
  print(ix,model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x7f750ffb7f60>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7f750ffb7e80>
2 <keras.layers.convolutional.Conv2D object at 0x7f750ffb7e48>
3 <keras.layers.normalization.BatchNormalization object at 0x7f750ff82710>
4 <keras.layers.core.Activation object at 0x7f750ff827f0>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7f750ff87978>
6 <keras.layers.pooling.MaxPooling2D object at 0x7f750ff8fcf8>
7 <keras.layers.convolutional.Conv2D object at 0x7f750ff397b8>
8 <keras.layers.normalization.BatchNormalization object at 0x7f750ff39ba8>
9 <keras.layers.core.Activation object at 0x7f750ff39ac8>
10 <keras.layers.convolutional.Conv2D object at 0x7f750ff3e550>
11 <keras.layers.normalization.BatchNormalization object at 0x7f750feeeac8>
12 <keras.layers.core.Activation object at 0x7f750fef40b8>
13 <keras.layers.convolutional.Conv2D object at 0x7f750fefa400>
14 <keras.layers.convolutional.Conv2D object at 0x7f750feaaf98>
15 <keras.lay

In [31]:
for ix in range(169):
  model.layers[ix].trainable = False

model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_new.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [32]:
hist = model_new.fit(X_train,Y_train,shuffle=True,batch_size=16,validation_split=0.2,epochs = 5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 646 samples, validate on 162 samples
Epoch 1/5
646/646 [==============================] - 210s 325ms/step - loss: 0.5175 - acc: 0.7988 - val_loss: 0.0998 - val_acc: 0.9753
Epoch 2/5
646/646 [==============================] - 208s 321ms/step - loss: 0.1105 - acc: 0.9628 - val_loss: 0.0719 - val_acc: 0.9753
Epoch 3/5
646/646 [==============================] - 208s 323ms/step - loss: 0.0888 - acc: 0.9690 - val_loss: 0.0812 - val_acc: 0.9630
Epoch 4/5
646/646 [==============================] - 207s 321ms/step - loss: 0.0905 - acc: 0.9706 - val_loss: 0.0620 - val_acc: 0.9753
Epoch 5/5
646/646 [==============================] - 207s 320ms/step - loss: 0.0746 - acc: 0.9768 - val_loss: 0.0932 - val_acc: 0.9568
